 ### Tutorial Neural Network - PyTorch
 
 Neste notebook é apresentado algumas observações e formas de implementação e ajustes que podem ser feitos em uma rede neural! A explicação completa pode ser encontrada neste link, junto com o código. 
 <link> https://machinelearningmastery.com/how-to-grid-search-hyperparameters-for-pytorch-models/ 

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
import torch.nn.init as init

In [2]:
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt(r'C:\Users\AnaLúciaLima/Documents/pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [3]:
# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x


In [4]:
# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adam,
    verbose=False
)

##### Neste exemplo, temos o gridSearchCV (técnica de otimização de hiperparâmetros) para ajustar o número de:
- batch_size é definido o número de amostras que serão propagadas pela rede antes de atualizar os parâmetros do modelo interno.
- max_epochs é definido como um hiperparâmetro que define o número de vezes que o algoritmo de aprendizado funcionará em todo o conjunto de dados de treinamento.

In [5]:
# define the grid search parameters
param_grid = {
    'batch_size': [10, 20, 40, 60, 80, 100],
    'max_epochs': [10, 50, 100]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [6]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.742188 using {'batch_size': 20, 'max_epochs': 100}
0.674479 (0.030978) with: {'batch_size': 10, 'max_epochs': 10}
0.662760 (0.007366) with: {'batch_size': 10, 'max_epochs': 50}
0.710938 (0.022097) with: {'batch_size': 10, 'max_epochs': 100}
0.677083 (0.013279) with: {'batch_size': 20, 'max_epochs': 10}
0.708333 (0.040133) with: {'batch_size': 20, 'max_epochs': 50}
0.742188 (0.003189) with: {'batch_size': 20, 'max_epochs': 100}
0.679688 (0.016573) with: {'batch_size': 40, 'max_epochs': 10}
0.683594 (0.030425) with: {'batch_size': 40, 'max_epochs': 50}
0.708333 (0.029463) with: {'batch_size': 40, 'max_epochs': 100}
0.653646 (0.022402) with: {'batch_size': 60, 'max_epochs': 10}
0.686198 (0.011201) with: {'batch_size': 60, 'max_epochs': 50}
0.709635 (0.032734) with: {'batch_size': 60, 'max_epochs': 100}
0.643229 (0.010253) with: {'batch_size': 80, 'max_epochs': 10}
0.696615 (0.038450) with: {'batch_size': 80, 'max_epochs': 50}
0.707031 (0.030758) with: {'batch_size': 80, 'max_epoch

#### Neural Network possue o algoritmo de otimização de treinamento 

- Nesta grid_search é definido alguns algoritmos de otimização, o algoritmos que apresentar melhor resultado é usado para treinar a rede. 

In [31]:
# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

In [17]:
# define the grid search parameters
param_grid = {
    'optimizer': [optim.SGD, optim.RMSprop, optim.Adagrad, optim.Adadelta,
                  optim.Adam, optim.Adamax, optim.NAdam],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.718750 using {'optimizer': <class 'torch.optim.adamax.Adamax'>}
0.645833 (0.003683) with: {'optimizer': <class 'torch.optim.sgd.SGD'>}
0.700521 (0.039623) with: {'optimizer': <class 'torch.optim.rmsprop.RMSprop'>}
0.686198 (0.014382) with: {'optimizer': <class 'torch.optim.adagrad.Adagrad'>}
0.582031 (0.052698) with: {'optimizer': <class 'torch.optim.adadelta.Adadelta'>}
0.713542 (0.032578) with: {'optimizer': <class 'torch.optim.adam.Adam'>}
0.718750 (0.029232) with: {'optimizer': <class 'torch.optim.adamax.Adamax'>}
0.710938 (0.045218) with: {'optimizer': <class 'torch.optim.nadam.NAdam'>}


#### Ajuste da taxa de aprendizagem e momentum 
A taxa de aprendizagem é de suma importancia para encontrar o equilíbrio  ideal entre a velocidade de convergência e a precisão. Já o momentum é utilizado para encontrar o equilíbrio ideal entre a velocidade de convergência e a estabilidade. 

In [32]:

# define the grid search parameters
param_grid = {
    'optimizer__lr': [0.001, 0.01, 0.1, 0.2, 0.3],
    'optimizer__momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.667969 using {'optimizer__lr': 0.001, 'optimizer__momentum': 0.0}
0.667969 (0.014616) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.0}
0.644531 (0.025315) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.2}
0.647135 (0.009744) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.4}
0.649740 (0.017566) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.6}
0.664062 (0.008438) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.8}
0.552083 (0.141826) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.9}
0.653646 (0.017566) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.0}
0.648438 (0.005524) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.2}
0.450521 (0.142719) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.4}
0.649740 (0.003683) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.6}
0.348958 (0.001841) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.8}
0.450521 (0.142719) with: {'optimizer__lr': 0.01, 'optimizer__

#### Ajuste da inicialização do peso da rede

A inicialização do peso é um fator extremamente importante nas redes neurais, pois pode afetar diretamente a velocidade de convergência e o desempenho do modelo. Para que o treinamento seja eficaz é importante definir inicialmente os valores dos pesos. 

In [20]:
# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self, weight_init=init.xavier_uniform_):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        # manually init weights
        weight_init(self.layer.weight)
        weight_init(self.output.weight)
 
    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x
 
# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

In [21]:
param_grid = {
    'module__weight_init': [init.uniform_, init.normal_, init.zeros_,
                           init.xavier_normal_, init.xavier_uniform_,
                           init.kaiming_normal_, init.kaiming_uniform_]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.705729 using {'module__weight_init': <function xavier_normal_ at 0x0000019FF39E0B80>}
0.348958 (0.001841) with: {'module__weight_init': <function uniform_ at 0x0000019FF39E05E0>}
0.555990 (0.136303) with: {'module__weight_init': <function normal_ at 0x0000019FF39E0670>}
0.658854 (0.012075) with: {'module__weight_init': <function zeros_ at 0x0000019FF39E08B0>}
0.705729 (0.024150) with: {'module__weight_init': <function xavier_normal_ at 0x0000019FF39E0B80>}
0.571615 (0.156000) with: {'module__weight_init': <function xavier_uniform_ at 0x0000019FF39E0AF0>}
0.678385 (0.018414) with: {'module__weight_init': <function kaiming_normal_ at 0x0000019FF39E0D30>}
0.541667 (0.137640) with: {'module__weight_init': <function kaiming_uniform_ at 0x0000019FF39E0CA0>}


#### Ajuste da função de ativação de neurônio

A função de ativação faz a transformação não-linear nos dados de entrada, tonando-o capaz de aprender e executar tarefas mais complexas.

In [24]:
class PimaClassifier(nn.Module):
    def __init__(self, activation=nn.ReLU):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = activation()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        # manually init weights
        init.kaiming_uniform_(self.layer.weight)
        init.kaiming_uniform_(self.output.weight)
 
    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x
 
# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

In [25]:
# define the grid search parameters
param_grid = {
    'module__activation': [nn.Identity, nn.ReLU, nn.ELU, nn.ReLU6,
                           nn.GELU, nn.Softplus, nn.Softsign, nn.Tanh,
                           nn.Sigmoid, nn.Hardsigmoid]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.694010 using {'module__activation': <class 'torch.nn.modules.activation.Softplus'>}
0.350260 (0.003683) with: {'module__activation': <class 'torch.nn.modules.linear.Identity'>}
0.691406 (0.019918) with: {'module__activation': <class 'torch.nn.modules.activation.ReLU'>}
0.582031 (0.164712) with: {'module__activation': <class 'torch.nn.modules.activation.ELU'>}
0.643229 (0.006639) with: {'module__activation': <class 'torch.nn.modules.activation.ReLU6'>}
0.692708 (0.020505) with: {'module__activation': <class 'torch.nn.modules.activation.GELU'>}
0.694010 (0.027866) with: {'module__activation': <class 'torch.nn.modules.activation.Softplus'>}
0.653646 (0.019225) with: {'module__activation': <class 'torch.nn.modules.activation.Softsign'>}
0.632812 (0.024910) with: {'module__activation': <class 'torch.nn.modules.activation.Tanh'>}
0.656250 (0.028705) with: {'module__activation': <class 'torch.nn.modules.activation.Sigmoid'>}
0.644531 (0.005524) with: {'module__activation': <class 'tor

#### Ajuste da regularização do Dropout

O Dropout é uma técnica regularizadora em deep learning que ajuda a previnir o overfitting. O dropout age descartando aleatoriamente ou ignorando alguns neurônios durante o treinamento. 

In [26]:
# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self, dropout_rate=0.5, weight_constraint=1.0):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        self.weight_constraint = weight_constraint
        # manually init weights
        init.kaiming_uniform_(self.layer.weight)
        init.kaiming_uniform_(self.output.weight)
 
    def forward(self, x):
        # maxnorm weight before actual forward pass
        with torch.no_grad():
            norm = self.layer.weight.norm(2, dim=0, keepdim=True).clamp(min=self.weight_constraint / 2)
            desired = torch.clamp(norm, max=self.weight_constraint)
            self.layer.weight *= (desired / norm)
        # actual forward pass
        x = self.act(self.layer(x))
        x = self.dropout(x)
        x = self.prob(self.output(x))
        return x
 
# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)
 

In [27]:
# define the grid search parameters
param_grid = {
    'module__weight_constraint': [1.0, 2.0, 3.0, 4.0, 5.0],
    'module__dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.717448 using {'module__dropout_rate': 0.1, 'module__weight_constraint': 1.0}
0.597656 (0.178523) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 1.0}
0.692708 (0.012075) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 2.0}
0.578125 (0.162755) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 3.0}
0.592448 (0.170935) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 4.0}
0.683594 (0.022999) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 5.0}
0.717448 (0.027126) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 1.0}
0.707031 (0.031412) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 2.0}
0.697917 (0.014382) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 3.0}
0.705729 (0.009207) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 4.0}
0.696615 (0.014382) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 5.0}
0.688802 (0.023

#### Ajustando o número de neurônios na camada oculta

In [28]:
class PimaClassifier(nn.Module):
    def __init__(self, n_neurons=12):
        super().__init__()
        self.layer = nn.Linear(8, n_neurons)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.output = nn.Linear(n_neurons, 1)
        self.prob = nn.Sigmoid()
        self.weight_constraint = 2.0
        # manually init weights
        init.kaiming_uniform_(self.layer.weight)
        init.kaiming_uniform_(self.output.weight)
 
    def forward(self, x):
        # maxnorm weight before actual forward pass
        with torch.no_grad():
            norm = self.layer.weight.norm(2, dim=0, keepdim=True).clamp(min=self.weight_constraint / 2)
            desired = torch.clamp(norm, max=self.weight_constraint)
            self.layer.weight *= (desired / norm)
        # actual forward pass
        x = self.act(self.layer(x))
        x = self.dropout(x)
        x = self.prob(self.output(x))
        return x
 
# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

In [29]:
# define the grid search parameters
param_grid = {
    'module__n_neurons': [1, 5, 10, 15, 20, 25, 30]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.712240 using {'module__n_neurons': 25}
0.649740 (0.003683) with: {'module__n_neurons': 1}
0.696615 (0.033502) with: {'module__n_neurons': 5}
0.675781 (0.035943) with: {'module__n_neurons': 10}
0.694010 (0.045592) with: {'module__n_neurons': 15}
0.696615 (0.010253) with: {'module__n_neurons': 20}
0.712240 (0.027126) with: {'module__n_neurons': 25}
0.701823 (0.034104) with: {'module__n_neurons': 30}
